In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import datetime as dt
from matplotlib import dates as mdates
import matplotlib
import pandas as pd
import corotools as ct
from IPython.display import display, Markdown
from tabulate import tabulate
import os

geopandasplot = True

# use geopandas to plot colored maps. geopandas, cartopy
# and suitable shape file (see below) have to be present
try:
    import geopandas
    import cartopy.crs as ccrs
    from mpl_toolkits.axes_grid1 import make_axes_locatable  # needs to be able to split colorba
except:
    print('Problems loading geopandas or cartopy')
    geopandasplot = False

population_file = os.environ['COVID19_SCRIPT_POPFILE']
jhu_data_dir = os.environ['COVID19_JHU_DIR']
datadir = os.environ['MYDATADIR']

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# some settings
tablesmininf=5000   # disregard countries with less cases for tables
plotsw = False   # plot weekly stats for countries?
plotswcont = True # plot weekly stats for continents?
tablesnum = 20 # number of entries for tables


In [ ]:
pops = ct.get_populations('~/Daten/worldpop.csv')
MDlist = ct.get_jhu_data(datadir = jhu_data_dir)
Md = MDlist[0].copy()
lastday = dt.datetime(*[int(x) for x in Md['US'][-1,0:3]])
lastdayasstr = lastday.strftime("%a, %d.%m.%Y")

metat = ct.get_metatable(MDlist, pops, [x for x in Md.keys()], ignore_list=ct.list_continents)
mt, mt2, mt4 = ct.get_metatable124(MDlist, pops, [x for x in Md.keys()], ignore_list=ct.list_continents)
speciallist = []#['World','North America','South America','Europe','Asia','Africa']
speciallist.sort()
speciallist.append('Austria')
speciallist.append('Belgium')
speciallist.append('Czechia')
speciallist.append('Denmark')
speciallist.append('France')
speciallist.append('Germany')
speciallist.append('Netherlands')
speciallist.append('Poland')
speciallist.append('Sweden')
speciallist.append('Switzerland')

In [ ]:
if geopandasplot:    
    # use a proper good shape file, e.g. from https://www.naturalearthdata.com
    try:
        world = geopandas.read_file(datadir+'maps/naturalearth/ne_10m_admin_0_countries.shp')
        # fix some bugs in naturalearth table: missing ISO3 codes
        world.loc[world['NAME'] == 'France', 'ISO_A3'] = 'FRA'
        world.loc[world['NAME'] == 'Norway', 'ISO_A3'] = 'NOR'
        world.loc[world['NAME'] == 'Somaliland', 'ISO_A3'] = 'SOM'
        world.loc[world['NAME'] == 'Kosovo', 'ISO_A3'] = 'RKS'
    except:
        print('problem reading shape file')
        geopandasplot=False
        
    contxylim = {'europe':[-22,40,35,70],
            'asia':[25,150,-20,60],
            'africa':[-25,55,-40,40],
            'north america':[-130,-60,5,60],
            'south america':[-90,-30,-60,15],
            'oceania':[100,180,-50,10],}


sources for data:
* Johns-Hopkins-University [JHU github](https://github.com/CSSEGISandData/COVID-19)
* [worldometers](https://www.worldometers.info) (populations)

remarks:
* due to strong daily fluctuations, my stats concentrate on weekly numbers/changes
* some numbers from JHU are different to 'officially' reported ones (e.g. France) - take it with a grain of salt

scripts for evaluation this data and creating this page:

* [xmhk/corotools](https://github.com/xmhk/corotools)

In [ ]:

def shortstat(mt, key, mininf=2000, factor = 1):
    x = mt[mt['cases']>mininf][key].to_numpy() * factor
    
    return np.array([np.median(x), np.mean(x), np.max(x)])

M = np.zeros((3,4))
M[:,0] = shortstat(mt, 'cases_perpop', factor=1000)
M[:,1] = shortstat(mt, 'cases_delta_perpop', factor=1e6)
M[:,2] = shortstat(mt, 'dead_perpop', factor=1e6)
M[:,3] = shortstat(mt, 'dead_delta_perpop', factor=1e6)

t = tabulate([['median',*M[0,:]],
               ['mean', *M[1,:]],
              ['max',*M[2,:]]], headers = ['cases /1k', 'dcases /1m', 'dead /1m', 'ddead /1m'],
             floatfmt='.1f',
            tablefmt='github')
display(Markdown('## tables: general cases info for countries > %.0f cases '%tablesmininf))
display(Markdown(t))

# tables: cases

remarks:
* 1st column: rank (-2 weeks, -4 weeks)
* **c(k)**: total cases (in thousands)
* **dc(k)**: last weeks increase in cases (in thousands)
* **c /1k**: total cases per population (per thousand)
* **dc /1m**: last weeks increase of cases per population (per million)
* **r(%)**: ratechange - relative change of new cases last week vs. the week before



In [ ]:
allkeys=[]
display(Markdown("remark: countries which had below %.0f cases two weeks ago are ranked at 200"%tablesmininf))


In [ ]:
cases_table = lambda sk :  ct.get_rank_table_from_df2(
        mt, mt2, mt4, sk,
        ['cases', 'cases_delta','cases_perpop', 'cases_delta_perpop', 'cases_ratechange'], 
        [1/1000,1/1000,1e3, 1e6,100],
        columnalias=['c(k)','dc(k)','c /1k', 'dc /1m', 'r(%)'],
         ascending=False, mininf=tablesmininf,
         num=tablesnum)

#for sk in ['cases', 'cases_delta', 'cases_perpop','cases_delta_perpop', 'cases_ratechange']:
for sk in ['cases_perpop','cases_delta_perpop', 'cases_ratechange']:
    display(Markdown("## %s week ending %s"%(sk,lastdayasstr)))
    if sk == 'cases':
        display(Markdown("remark: countries which had below %.0f cases two weeks ago are ranked at 200"%tablesmininf))
    display(Markdown(cases_table(sk)))
    ll = ct.sort_df_and_get_keys(mt[mt['cases']>tablesmininf],sk,num=tablesnum)
    for x in ll:
        if x not in allkeys:
            allkeys.append(x)

# tables: dead

remarks:
* 1st column: rank (-2 weeks, -4 weeks)
* **d(k)**: total dead (in thousands)
* **dd(k)**: last weeks increase in dead (in thousands) 
* **d /1k**: total dead per population (per thousand)
* **dd /1m**: last weeks increase of dead per population (per million)
* **r(%)**: ratechange - relative change of new dead last week vs. the week before

In [ ]:
dead_table = lambda sk :  ct.get_rank_table_from_df2(mt,mt2,mt4
        , sk,
        ['dead', 'dead_delta','dead_perpop', 'dead_delta_perpop', 'dead_ratechange'], 
        [1/1000,1/1000,1e6, 1e6,100],
        columnalias=['d(k)','dd(k)','d /1m', 'dd /1m', 'r(%)'],
         ascending=False, mininf=tablesmininf,
         num=tablesnum)

#for sk in ['dead', 'dead_delta', 'dead_perpop','dead_delta_perpop']:
for sk in ['dead_perpop','dead_delta_perpop']:
    
    display(Markdown("## %s week ending %s"%(sk,lastdayasstr)))
    display(Markdown(dead_table(sk)))
    ll = ct.sort_df_and_get_keys(mt[mt['cases']>tablesmininf],sk,num=tablesnum)
    for x in ll:
        if x not in allkeys:
            allkeys.append(x)


# continents

In [ ]:
if geopandasplot:
    newcolumn = 'weeklycasesper1m'
    mtcolumn = 'cases_delta_perpop'
    modfactor = 1e6

    world['mtknown']=False
    world[newcolumn] = 0    
    # add some info to the world geopandas dataframe for plotting colored maps
    for key in mt['iso3'].to_list():
        if key in world['ISO_A3'].to_list():
            indx = world.loc[lambda df: df['ISO_A3']==key].index[0]
            newval = mt.loc[lambda df:df['iso3']==key][mtcolumn].to_numpy()[0] * modfactor
            contval = mt.loc[lambda df:df['iso3']==key]['continent'].to_numpy()[0]
            world.at[indx,newcolumn] = newval
            world.at[indx,'continent'] = contval
            world.at[indx,'mtknown'] = True
        else:
            pass#print('%s not in keys'%key)
     



In [ ]:
for k in ct.list_continents:
    display(Markdown("\n\n### %s"%k))
    if geopandasplot:
        fig = plt.figure(dpi=100, figsize=(7,6))
        ax = plt.subplot(111)
        if k !='World':
            cont = k.lower()
            contdf =  world[world['continent']==cont]
            fig = plt.figure(dpi=100)
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="2%", pad=0.1)
            ax =contdf.plot(newcolumn, ax=ax,vmin = 0,legend=True,cax=cax,cmap='inferno_r')
            ax.set_xlim(contxylim[cont][0:2])
            ax.set_ylim(contxylim[cont][2:4])
        else:
            contdf =  world[world['mtknown']==True]
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="2%", pad=0.1)
            ax =contdf.plot(newcolumn, ax=ax,vmin = 0,legend=True,cax=cax,cmap='inferno_r')
            ax.set_ylim([-70,70])
            ax.set_xticks([])
            ax.set_yticks([])
        ax2 = world.boundary.plot(ax = ax,lw=0.2,)
        ax.set_title('weekly new cases per 1 m %s'%lastdayasstr)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.show()
        
    if plotswcont:
        f, ax1, ax2 = ct.plot_rates(MDlist[0], MDlist[1], k)
        plt.show()
        

    display(Markdown(ct.get_country_weekly_stat_table(MDlist, pops, k, tableformat='github')))

In [ ]:
# show countries which have higher infection rates per population than 80% of the considered countries
mtB = mt[mt['cases']>tablesmininf]
qt = np.quantile(mtB['cases_delta_perpop'].to_numpy(),0.8)
mtC = mtB[mtB['cases_delta_perpop']>qt]
mtD = mtC.sort_values(by='key')
len(mtC['key'])

severekeys = mtD['key'].to_list()

display(Markdown('# countries > %d cases and > %.1f new cases per million last week'%(tablesmininf, qt*1e6)))

for k in severekeys:
    display(Markdown("\n\n### %s"%k))
    if plotsw:
        f, ax1, ax2 = ct.plot_rates(MDlist[0], MDlist[1], k)
        plt.show()
    display(Markdown(ct.get_country_weekly_stat_table(MDlist, pops, k, tableformat='github')))


# additional countries


In [ ]:
for k in speciallist:
    if k not in allkeys:
        allkeys.append(k)

allkeys.sort()
speciallist.sort()
for k in speciallist:
    display(Markdown("\n\n### %s"%k))
    if plotsw:
        f, ax1, ax2 = ct.plot_rates(MDlist[0], MDlist[1], k)
        plt.show()
    display(Markdown(ct.get_country_weekly_stat_table(MDlist, pops, k, tableformat='github')))